In [13]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pprint 
from pymongo import MongoClient
from selenium import webdriver
from splinter import Browser

In [2]:
# scrape news site for title and paragraph text, save off to variables
news_url = "https://mars.nasa.gov/news/"
driver = webdriver.Firefox()
driver.get(news_url)
driver.implicitly_wait(10)
html = driver.page_source
news_soup = bs(html, "html.parser")

news_title = (news_soup.find("div", class_="list_text")).find("a").text
print(news_title)

news_paragraph = news_soup.find("div", class_="article_teaser_body").text
print(news_paragraph)

NASA to Reveal Name of Its Next Mars Rover
After a months-long contest among students to name NASA's newest Mars rover, the agency will reveal the winning name — and the winning student — this Thursday. 


In [3]:
# find featured Mars image from JPL, save full-size image url
# note: this url doesn't seem to pull up a Mars image any longer
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
base_jpl_url = "https://www.jpl.nasa.gov"
driver = webdriver.Firefox()
driver.get(jpl_url)
html = driver.page_source
img_soup = bs(html, "html.parser")

#build url from sytle tag
image_url  = img_soup.find("article")["style"].replace('background-image: url(','').replace(');', '')[1:-1]
featured_image_url = base_jpl_url + image_url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16711-1920x1200.jpg


In [15]:
# scrape latest mars weather from twitter
mars_wx_url = "https://twitter.com/MarsWxReport"
r = requests.get(mars_wx_url)
html = r.text
wx_soup = bs(html, "html.parser")
mars_weather = wx_soup.find_all("div", class_="js-tweet-text-container")[0].text
print(mars_weather)


InSight sol 450 (2020-03-02) low -93.5ºC (-136.4ºF) high -10.4ºC (13.3ºF)
winds from the SSW at 5.5 m/s (12.4 mph) gusting to 20.6 m/s (46.1 mph)
pressure at 6.30 hPapic.twitter.com/82lzRqibcC



In [16]:
# scrape Mars facts website, display as html table str using pandas
mars_facts_url = "https://space-facts.com/mars/"
mars_df = pd.read_html(mars_facts_url)[0]

##  print html string
mars_facts_html_str = mars_df.to_html()
pprint.pprint(mars_facts_html_str)

## render html
mars_facts_html = mars_df.to_html("mars_facts.html")

('<table border="1" class="dataframe">\n'
 '  <thead>\n'
 '    <tr style="text-align: right;">\n'
 '      <th></th>\n'
 '      <th>0</th>\n'
 '      <th>1</th>\n'
 '    </tr>\n'
 '  </thead>\n'
 '  <tbody>\n'
 '    <tr>\n'
 '      <th>0</th>\n'
 '      <td>Equatorial Diameter:</td>\n'
 '      <td>6,792 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>1</th>\n'
 '      <td>Polar Diameter:</td>\n'
 '      <td>6,752 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>2</th>\n'
 '      <td>Mass:</td>\n'
 '      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>3</th>\n'
 '      <td>Moons:</td>\n'
 '      <td>2 (Phobos &amp; Deimos)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>4</th>\n'
 '      <td>Orbit Distance:</td>\n'
 '      <td>227,943,824 km (1.38 AU)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>5</th>\n'
 '      <td>Orbit Period:</td>\n'
 '      <td>687 days (1.9 years)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>6</th>\n'
 '      <td

AttributeError: 'NoneType' object has no attribute 'drop'

In [12]:
# scrape USGS Astrogeology site for high res photos of hemispheres
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser = Browser('firefox')
browser.visit(usgs_url)

#create list of titles and urls to loop through
hemisphere_list = [(a.text, a['href']) for a in browser.find_by_css('div[class="description"] a')]

hemisphere_dict = []
for title, hemisphere_url in hemisphere_list:
    temp_dict = {}
    temp_dict["title"] = title
    browser.visit(hemisphere_url)
    temp_dict["img_url"] = browser.find_by_css('img[class="wide-image"]')['src']
    hemisphere_dict.append(temp_dict)
pprint.pprint(hemisphere_dict)

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]
